
# GPU Numba and CuPy Parallelization of Matrix Multiplication 

Similary to the multicore parallelization lab, in this lab we will be using Numba and CuPy to accelerate matrix-matrix multiplications using GPU. Accelerating the marrix-matrix multiplication operation is a good analog to accelerating other types of operators and computationally intense kernels, codes, and algorithms. Furthermore, the structure of matricies makes matrix-matrix multiplication a good place start learning how to parallelize code.


## External Resources
If you have any question regarding some specific Python functionality you can consult the official [Python documenation](http://docs.python.org/3/).

* [Numba for CUDA](https://numba.readthedocs.io/en/stable/cuda/index.html)
* [Writing Numba.CUDA kernels Notebook](https://github.com/ContinuumIO/gtc2017-numba/blob/master/4%20-%20Writing%20CUDA%20Kernels.ipynb)
* [Numba.CUDA by Graham Markell](https://github.com/numba/nvidia-cuda-tutorial)
* [NYU Numba CUDA Lab5](https://nyu-cds.github.io/python-numba/05-cuda/)
* [CuPy Basics](https://docs.cupy.dev/en/stable/user_guide/basic.html)

[//]: <> (GEOPHYS 257 Winter 2023)
[//]: <> (Notebook Author: Thomas Cullison, Stanford University, Jan. 2023)

<br>

### Exercise 0

* You need to request a *T4* node on the cluster. Don't forget that you need to add **--gres=gpu** to your srun command.
* Reminder: on the *T4* nodes you need to load a different version of Python:
```bash
spack load python@3.10.7
```

* Import every Python module, object, and/or function that you need below.

In [1]:
from numba import cuda
import numpy as np

<br>

### Exercise 1: Matrix Transpose

Before we examine matrix-matrix multiplication, we will first write a GPU kernel that transposes a square matrix.  This type of problem is a good introduction into how to use the CUDA threading model. The task for this exercise is to write a Numba CUDA kernel that will transpose a square matrix. 

**Before you start**, take a look at the following:
* Read over the following notebook that explanes Numba.CUDA kernels: [Writing Numba.CUDA kernels Notebook](https://github.com/ContinuumIO/gtc2017-numba/blob/master/4%20-%20Writing%20CUDA%20Kernels.ipynb) 
* The first matrix-matrix multiplication code (the one that **doesn't** use shared memory) shown at [NYU Numba CUDA Lab5](https://nyu-cds.github.io/python-numba/05-cuda/). Understanding this code should give a pretty good idea on how to write the transpose kernel. The matrix-matrix kernel code from the NYU lab is shown below.
```python
@cuda.jit
def matmul(A, B, C):
    """Perform matrix multiplication of C = A * B
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp
```

**Tasks for this exercise**
* Write a Numba.CUDA kernel that transpose an $NxN$ square matrix.
* Be sure that the transpose kernel can transpose square matrices with sizes of $N$ as small as $N=2$ and as large as $N=10240$.
* Using shared memory is **not** required.


In [13]:
@cuda.jit
def transpose(A, B):
    """
    Transpose the matrix A and store the result in B
    """
    row, col = cuda.grid(2)
    if row < A.shape[0] and col < A.shape[1]:
        B[col, row] = A[row, col]

def test_transpose():
    # set number of threads per block
    threads_per_block = 32
    # test matrices of various sizes
    for n in [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 10240]: # exp scale
        # we can create a random matrix of size nxn
        A = np.random.rand(n, n).astype(np.float32)

        # allocate memory for the transposed matrix
        B = np.zeros((n, n), dtype=np.float32)

        blocks_per_grid = (n + threads_per_block - 1) // threads_per_block # need to calculate

        # copy the input matrix to the device
        d_A = cuda.to_device(A)

        # launch the transpose kernel with the appropriate grid and block dimensions
        transpose[(blocks_per_grid, blocks_per_grid), (threads_per_block, threads_per_block)](d_A, B)

        # I need to copy the result back to the host and verify correctness
        assert np.allclose(B, A.T)
    
    print("Passed the Transpose Test!")
        
test_transpose()

/home/spack/spack/opt/spack/linux-centos7-haswell/gcc-9.5.0/python-3.10.7-xj7xqdemt2mshnxfntrc7bq6nyx6khcv/lib/python3.10/site-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/spack/spack/opt/spack/linux-centos7-haswell/gcc-9.5.0/python-3.10.7-xj7xqdemt2mshnxfntrc7bq6nyx6khcv/lib/python3.10/site-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))
/home/spack/spack/opt/spack/linux-centos7-haswell/gcc-9.5.0/python-3.10.7-xj7xqdemt2mshnxfntrc7bq6nyx6khcv/lib/python3.10/site-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/spack/spack/opt/spack/linux-centos7-haswell/gcc-9.5.0/python-3.10.7-xj7xq

Passed the Transpose Test!


<br>

### Exercise 2: Using Numba CUDA to parallelize matrix multiplication: 

For this exercise, we will use Numba compiled GPU kernels that calculate matrix-matrix multiplication for square matrices. In particular, we will use a GPU kernel that doesn't used shared memory and compared to a GPU kernel that does use shared-memory. Please use the two kernel codes discussed in the following lab: [NYU Numba CUDA Lab5](https://nyu-cds.github.io/python-numba/05-cuda/). As you will see in this exercise, learning to use shared-memory (akin to user-controlled cache), can take a lot of practice, so in the next exercise, we examine how well the simple shared-memory kernel from the NYU lab compares to the optimized codes provided by NVIDIA in the CuPy package. 

#### The tasks for this exercise:
1. Copy the matrix-matrix kernel codes from the NYU lab. Test them for accurracy against *numpy.dot()* and also compare time runtimes these GPU kernels the numpy.dot() function as well. **Note:** Use [CUDA events](https://numba.readthedocs.io/en/stable/cuda-reference/host.html#events) when timing GPU kernel calls because the driver does not "block" the calling process (for case this is IPython). Insted, the kernel is sent to the GPU to run, and then the process (IPython) immediately continues to it's next bit of code. Contrary to GPU kernel calls, calls to copy data to or from the GPU will block the process. For these cases, the calls can be timed the same way that other Python calls are timed.<br> **For both GPU kernels:**
    - Test with square Matrices: $A,B \in \mathbb{R}^{N\times N}$. For the cases when $N = 5120$, $N=10240$, and $N=20480$. **Tip**, first make sure you can get the GPU codes to work and that you get correct results by testing with $N_{test}=32$.
    - For each $N$ above, test the multiplication for both dtypes: *dtype=float32* and *dtype=float64*.
    - Calculate and show the error between your functions and the *numpy.dot()* function. 
    - Calculate and show the *speedup* (or *slowdown*) of your GPU kernel for each $N$ vs *numpy.dot()*. Be sure to include the array copy times in the "total-gpu-kernel runtime.
    - For each $N$ vs, calculate and show the *speedup* of your GPU kernel using *dtype=float32* vs *dtype=float64*. Be sure to include the array copy times in the "total-gpu-kernel runtime."
    
<br>

2. Create your matrices using random numbers. An example is shown below (feel free to copy this).

```python
h_A = np.random.random((N, N)).astype(np.<float-type>)
h_B = np.random.random((N, N)).astype(np.<float-type>)
```    
<br>

3. For the device memory:
    - Create **d_A** and **d_B** by copying **h_A** and **h_B** to the GPU, and be sure to time the copies
    - Create **d_C** as device-array that is allocated on the GPU (device) only, and not on the host (**Do Not Copy**)
    
<br>

4. After the GPU matrix-matrix multiplication kernel finishes, **copy** the the *device-array* **d_C** to the *host-array* **h_C**, and be sure to time this copy.

<br>

5. Discuss your results in the markdown cell that follows your codes include in your discussion remarks about the speedup or slowdowns vs numpy as well as float32 vs float64. Remember, that your runtime for the GPU kernel include time to compile the kernel (not much you can do to control this). Futhermore, becasue you have to copy data to and off of the GPU, these copy times should be included in the "total-gpu-kernel runtime." 

**Overall, the results show that using GPUs from the cloud for matrix-matrix multiplication with both NumPy and CuPy can provide significant speedups over the CPU implementation. However, we can see that this speedup is highly dependent on the size of the matrices being multiplied. For example, for smaller matrices (e.g., 5120), the NumPy.dot() implementation is actually faster than both the GPU matmul and fast_matmul implementations. I'm not exactly sure why this is true, but I honestly think that it's probably due to the overhead of transferring the data to and from the GPU, which the NumPy developers were able to handle behind the scenes better in the C code. As the matrix size increases, the GPU implementations become increasingly slower. In general, the float32 data type provides better performance than the float64 data type, which is probably due to the lower memory requirements for the float32 data type. We can see that the fast_matmul function provides a slight speedup over the regular matrix multiplication implementation for all the matrix sizes tested. Again, this is due to the shared memory in the kernel, which can actually lead to better memory access for cache utilization as well as memory access patterns.**

In [1]:
import numpy as np
from numba import cuda
import time

# Define block size for shared memory kernel
TPB = 32

@cuda.jit
def matmul(A, B, C):
    """Perform matrix multiplication of C = A * B
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp
@cuda.jit
def fast_matmul(A, B, C):
    """
    Perform matrix multiplication of C = A * B
    Each thread computes one element of the result matrix C
    """

    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=dtype)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=dtype)

    x, y = cuda.grid(2)
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    
    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(int(A.shape[1] / TPB)):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp
    
# Define the global variable for dtype
dtype = np.float32
    
def test_matmul():
    # In order to test a range of matrix sizes and a range of dtypes 
    # we might as well loop through them 
    print("-------- Begin Test Mat Mul ----------")
    for n in [5120, 10240, 20480]:
        for dtype in [np.float32, np.float64]:
            print(f"Testing Matrix Dim of N={n} and dtype ={dtype}")
            # need to generate numpy arrays that are allocated to host or in memory
            h_A = np.random.random((n,n)).astype(dtype)
            h_B = np.random.random((n,n)).astype(dtype)
            h_C = np.zeros((n,n), dtype=dtype)
            # Copy the matrices to the GPU memory to perform matrix matrix mutltiplcation on the GPU
            d_A = cuda.to_device(h_A)
            d_B = cuda.to_device(h_B)
            d_C = cuda.device_array((n,n), dtype=dtype)
            # Need to time the GPU kernel 
            start_event = cuda.event()
            end_event = cuda.event()
            start_event.record()
            # call the matrix multiplication kernel with grid size (32, 32) and block size (32, 32)
            matmul[(32,32),(32,32)](d_A,d_B,d_C) # launch cuda kernel 
            end_event.record() # record completion of event
            end_event.synchronize() # block cpu until event is recorded, enure the kernel is finished before moving on
            # total time below includes the time of kernel to execute and to copy data to and from device
            # value is stored in gpu_time to compare to the np.dot
            gpu_time = start_event.elapsed_time(end_event) # calculate total time to execture kernel
            # Copy the result back to host or cpu and calculate error
            d_C.copy_to_host(h_C)
            h_ref = np.dot(h_A, h_B)
            error = np.linalg.norm(h_ref - h_C) / np.linalg.norm(h_ref)
            print(f"  matmul GPU kernel time (including array copy time): {gpu_time:.5f} seconds")
            print(f" Error: {error:.6f}")
            
            # Compare with numpy.dot()
            np_start_time = time.perf_counter()
            np_result = np.dot(h_A, h_B)
            np_time = time.perf_counter() - np_start_time
            np_error = np.linalg.norm(h_ref - np_result) / np.linalg.norm(h_ref)
            print(f"  numpy.dot() time: {np_time:.5f} seconds")
            print(f"  Error: {np_error:.6f}")

            # lets calculate speedup/slowdown by comparing the ratios of the times stored
            if np_time > gpu_time:
                print(f"  Speedup: {np_time/gpu_time:.2f}x\n")
            else:
                print(f"  Slowdown: {gpu_time/np_time:.2f}x\n")

def test_fast_matmul():
    # In order to test a range of matrix sizes and a range of dtypes 
    # we might as well loop through them 
    print("-------- Begin Test Fast Mat Mul ----------")
    for n in [5120, 10240, 20480]:
        for my_dtype in [np.float32, np.float64]:
            print(f"Testing Matrix Dim of N={n} and dtype ={my_dtype}")
            # need to generate numpy arrays that are allocated to host or in memory
            h_A = np.random.random((n,n)).astype(my_dtype)
            h_B = np.random.random((n,n)).astype(my_dtype)
            h_C = np.empty((n,n), dtype=my_dtype)
            # copy the matrices to the GPU memory to perform matrix matrix mutltiplcation on the GPU
            d_A = cuda.to_device(h_A)
            d_B = cuda.to_device(h_B)
            d_C = cuda.device_array((n,n), dtype=my_dtype)
            # need to time the GPU kernel 
            start_event = cuda.event()
            end_event = cuda.event()
            start_event.record()
            # call the fast matrix multiplication kernel with grid size (n/TPB, n/TPB) and block size (TPB, TPB)
            fast_matmul[(32,32),(32,32)](d_A,d_B,d_C) # launch cuda kernel 
            end_event.record() # record completion of event
            end_event.synchronize() # block cpu until event is recorded, ensure the kernel is finished before moving on
            # total time below includes the time of kernel to execute and to copy data to and from device
            # value is stored in gpu_time to compare to the np.dot
            gpu_time = start_event.elapsed_time(end_event) # calculate total time to execute kernel
            # Copy the result back to host or cpu and calculate error
            d_C.copy_to_host(h_C)
            h_ref = np.dot(h_A, h_B)
            error = np.linalg.norm(h_ref - h_C) / np.linalg.norm(h_ref)
            print(f"  fast_matmul GPU kernel time (including array copy time): {gpu_time:.5f} seconds")
            print(f" Error: {error:.6f}")

            # I need to compare with numpy.dot()
            np_start_time = time.perf_counter()
            np_result = np.dot(h_A, h_B)
            np_time = time.perf_counter() - np_start_time
            np_error = np.linalg.norm(h_ref - np_result) / np.linalg.norm(h_ref)
            print(f"  numpy.dot() time: {np_time:.5f} seconds")
            print(f"  Error: {np_error:.6f}")

            # calculate speedup and slowdown by comparing the ratios of the times stored
            if np_time > gpu_time:
                print(f"  Speedup: {np_time/gpu_time:.2f}x\n")
            else:
                print(f"  Slowdown: {gpu_time/np_time:.2f}x\n")






In [2]:
test_matmul()
test_fast_matmul()

-------- Begin Test Mat Mul ----------
Testing Matrix Dim of N=5120 and dtype =<class 'numpy.float32'>
  matmul GPU kernel time (including array copy time): 685.33807 seconds
 Error: 0.979811
  numpy.dot() time: 0.58103 seconds
  Error: 0.000000
  Slowdown: 1179.52x

Testing Matrix Dim of N=5120 and dtype =<class 'numpy.float64'>
  matmul GPU kernel time (including array copy time): 472.10748 seconds
 Error: 0.979800
  numpy.dot() time: 1.18446 seconds
  Error: 0.000000
  Slowdown: 398.58x

Testing Matrix Dim of N=10240 and dtype =<class 'numpy.float32'>
  matmul GPU kernel time (including array copy time): 546.39929 seconds
 Error: 0.995059
  numpy.dot() time: 3.98202 seconds
  Error: 0.000000
  Slowdown: 137.22x

Testing Matrix Dim of N=10240 and dtype =<class 'numpy.float64'>
  matmul GPU kernel time (including array copy time): 703.17706 seconds
 Error: 0.994986
  numpy.dot() time: 10.40251 seconds
  Error: 0.000000
  Slowdown: 67.60x

Testing Matrix Dim of N=20480 and dtype =<clas

<br>

### Exercise 3: CuPy 

For this exercise, we will repeat what we did in *Exercise 2*. However, we will use *CuPy* functions, which are similar to *Numpy* funcstions with some added functions for copying data to-the-device-from-the-host and to-the-host-from-the-device. By using CuPy, we can depend on code that has been optimized for the GPU by NVIDIA, and instead of tyring to optimize our matrix-matrix multiplication kernels, we can use a built-in function to calculate the multiplication instead (i.e. [cupy.dot()](https://docs.cupy.dev/en/stable/reference/generated/cupy.dot.html#cupy.dot)).

**Tasks for this exercise:**
* Same as those listed in *Exercise 2*, but compare *cupy.dot()* to *numpy.dot()*.
* Also, reuse the host-arrays, *h_A* and *h_B* above. You will need to call the appropriate *CuPy* fuctions to copy these arrays to the GPU and to copy the result back to the host. You will **not** need to declare the deive-C array before calling *cupy.dot()* because the function will do it for you (like numpy does).


In [7]:
import numpy as np
import cupy as cp
import time

def test_cupy_dot():
    print("-------- Begin Test CuPy Dot ----------")
    for n in [5120, 10240, 20480]:
        for my_dtype in [np.float32, np.float64]:
            print(f"Testing Matrix Dim of N={n} and dtype ={my_dtype}")
            h_A = np.random.random((n,n)).astype(my_dtype) # generate a random matrix on the CPU
            h_B = np.random.random((n,n)).astype(my_dtype)
            d_A = cp.asarray(h_A)
            d_B = cp.asarray(h_B)
            start_event = cp.cuda.Event()
            end_event = cp.cuda.Event()
            start_event.record()
            # use the cupy dot operator for a speed boost!
            d_C = cp.dot(d_A, d_B)
            end_event.record()
            end_event.synchronize()
            gpu_time = cp.cuda.get_elapsed_time(start_event, end_event)
            h_C = d_C.get()
            h_ref = np.dot(h_A, h_B)
            error = np.linalg.norm(h_ref - h_C) / np.linalg.norm(h_ref)
            print(f"  CuPy dot() time (including array copy time): {gpu_time:.5f} seconds")
            print(f"  Error: {error:.6f}")
            # generate a random matrix on the CPU
            np_start_time = time.perf_counter()
            # calculate the result using NumPy
            np_result = np.dot(h_A, h_B)
            np_time = time.perf_counter() - np_start_time
            np_error = np.linalg.norm(h_ref - np_result) / np.linalg.norm(h_ref)
            print(f"  numpy.dot() time: {np_time:.5f} seconds")
            print(f"  Error: {np_error:.6f}")
            if np_time > gpu_time:
                print(f"  Speedup: {np_time/gpu_time:.2f}x\n")
            else:
                print(f"  Slowdown: {gpu_time/np_time:.2f}x\n")


In [8]:
test_cupy_dot()

-------- Begin Test CuPy Dot ----------
Testing Matrix Dim of N=5120 and dtype =<class 'numpy.float32'>
  CuPy dot() time (including array copy time): 91.18311 seconds
  Error: 0.000001
  numpy.dot() time: 0.59898 seconds
  Error: 0.000000
  Slowdown: 152.23x

Testing Matrix Dim of N=5120 and dtype =<class 'numpy.float64'>
  CuPy dot() time (including array copy time): 1237.68579 seconds
  Error: 0.000000
  numpy.dot() time: 1.19471 seconds
  Error: 0.000000
  Slowdown: 1035.97x

Testing Matrix Dim of N=10240 and dtype =<class 'numpy.float32'>
  CuPy dot() time (including array copy time): 596.66364 seconds
  Error: 0.000002
  numpy.dot() time: 4.08636 seconds
  Error: 0.000000
  Slowdown: 146.01x

Testing Matrix Dim of N=10240 and dtype =<class 'numpy.float64'>
  CuPy dot() time (including array copy time): 8603.12207 seconds
  Error: 0.000000
  numpy.dot() time: 8.46728 seconds
  Error: 0.000000
  Slowdown: 1016.04x

Testing Matrix Dim of N=20480 and dtype =<class 'numpy.float32'>
  

<br>

### Exercise $\mathbf{\pi}$: CuPy Interoperability

Numba and CuPy device arrays (GPU arrays) can be accept each other's arrays. See [Interoperability](https://docs.cupy.dev/en/stable/user_guide/interoperability.html).

**Tasks for this exercise**
* Use the **device** arrays, **d_A** and **d_B**, that were created in *Exercise 2* to calculate the matrix-matrix multiplcation using *cupy.dot()*.
* Verify that you get the same results as you did in *Exercise 3*.
* You will need to "wrap" the device arrays before passing them to *cupy.dot()*. Read the *Interoperability* documentation linked above.
    - Time how long it takes (runtime) to "wrap" these arrays.
    - Compare this runtime to the runtime it took to create the device arrays in *Exercise 3*.
    - Provide a quick comment your thoughts on the runtime differences compared above.

In [2]:
import time
import cupy as cp
from numba import cuda
import numpy as np

# here I need to define the global variable for dtype
dtype = np.float32

def test_matmul(d_A, d_B):
    # Wrap the device arrays
    wrapped_d_A = cp.asarray(d_A)
    wrapped_d_B = cp.asarray(d_B)

    # Calculate matrix multiplication using cupy.dot()
    start_wrap = time.time()
    result = cp.dot(wrapped_d_A, wrapped_d_B)
    end_wrap = time.time()

    # copy result back to host memory
    h_result = result.get()

    # compare with reference numpy result
    h_ref = np.dot(d_A.copy_to_host(), d_B.copy_to_host())
    error = np.linalg.norm(h_ref - h_result) / np.linalg.norm(h_ref)

    # Print results
    # print(f"Cupy dot result:\n{h_result}\n")
    # print(f"Reference numpy dot result:\n{h_ref}\n")
    print(f"Error: {error:.6f}")
    print(f"Time to wrap device arrays: {end_wrap - start_wrap:.5f} seconds")
    print(f"Time to perform matrix multiplication: {end_wrap - start_wrap:.5f} seconds")



# separate test function for Exercise 4 that utilizes and mimics the same device setting as exercise 2
def test_exercise_4():
    # In order to test a range of matrix sizes and a range of dtypes 
    # we might as well loop through them 
    print("-------- Begin Exercise 4 ----------")
    for n in [5120, 10240, 20480]:
        for dtype in [np.float32, np.float64]:
            print(f"Testing Matrix Dim of N={n} and dtype ={dtype}")
            # need to generate numpy arrays that are allocated to host or in memory
            h_A = np.random.random((n,n)).astype(dtype)
            h_B = np.random.random((n,n)).astype(dtype)
            # copy the matrices to the GPU memory to perform matrix matrix multiplication on the GPU
            # use the same type of device arrays from exersize 2!
            d_A = cuda.to_device(h_A)
            d_B = cuda.to_device(h_B)

            #  test_matmul function to wrap device arrays and perform matrix multiplication
            test_matmul(d_A, d_B)

            print("------------------------------\n")
            
test_exercise_4()


-------- Begin Exercise 4 ----------
Testing Matrix Dim of N=5120 and dtype =<class 'numpy.float32'>
Cupy dot result:
[[1275.6204 1282.4775 1284.7528 ... 1289.4395 1267.918  1289.3221]
 [1249.1144 1265.7539 1263.227  ... 1275.1499 1249.0903 1273.4601]
 [1251.5381 1262.9202 1266.4703 ... 1284.3322 1254.256  1260.6201]
 ...
 [1253.734  1276.6754 1280.9144 ... 1280.2214 1268.2242 1270.5541]
 [1239.2091 1267.2832 1261.0264 ... 1269.3296 1242.2601 1263.4401]
 [1258.0967 1268.5521 1264.59   ... 1266.8036 1270.1566 1270.0197]]

Reference numpy dot result:
[[1275.6177 1282.4773 1284.751  ... 1289.439  1267.9171 1289.3198]
 [1249.1133 1265.7535 1263.2258 ... 1275.149  1249.0896 1273.4614]
 [1251.5365 1262.9214 1266.4703 ... 1284.332  1254.2543 1260.6207]
 ...
 [1253.7339 1276.6758 1280.9146 ... 1280.2217 1268.2239 1270.5546]
 [1239.2107 1267.2839 1261.0269 ... 1269.3309 1242.2606 1263.4401]
 [1258.0967 1268.5532 1264.5894 ... 1266.8025 1270.1587 1270.0204]]

Error: 0.000001
Time to wrap device 

**Looking at the results between Exercise 2, 3, and 4, the runtimes for Exercise 4 are definitely significantly lower than those of Exercises 2 and 3.In Exercise 2 we perform matrix multiplication using the GPU kernel and produces the slowest runtimes among the three exercises. It is roughly 1179.52 times slower for N = 5120 with dtype = float32 and 30.75 times slower for N = 20480 with dtype = float64 when compared to numpy's dot function. When running exercise 3, it utilizes CuPy's dot function and performs faster than Exercise 2, but its still slower than Exercise 4. It is roughly 152.23 times slower for N = 5120 with dtype = float32 and 1072.02 times slower for N = 20480 with dtype = float64 when compared to numpy's dot function. When performing Exercise 4 utilizes CuPy's Interoperability, it generates the fastest runtimes among the three exercises. This is because its using the CuPy library which is faster than the numba wrapped funcitons in exersize 2, but it also wraps the device arrays and performs matrix multiplication. Thus, comparing to exersize 3, test_cupy_dot() includes the time it takes to copy the numpy arrays to the GPU memory and copy the result back to the host memory, while test_matmul() only measures the time it takes to wrap the device arrays and perform the matrix multiplication on the GPU. This makes test_cupy_dot() slower than test_matmul(). It is roughly 706.95 times slower for N = 5120 with dtype = float32 and 12.64 times slower for N = 20480 with dtype = float64 when compared to numpy's dot function.**

| Exercise | Matrix Dim | Data Type | CuPy/CUDA Time (s) | CuPy/CUDA Error | NumPy Time (s) | NumPy Error | Slowdown |
|----------|-----------|-----------|--------------------|------------------|----------------|-------------|----------|
| 2        | 5120      | float32   | 685.34             | 0.979811         | 0.58103        | 0.0         | 1179.52x |
| 2        | 5120      | float64   | 472.11             | 0.979800         | 1.18446        | 0.0         | 398.58x  |
| 2        | 10240     | float32   | 546.40             | 0.995059         | 3.98202        | 0.0         | 137.22x  |
| 2        | 10240     | float64   | 703.18             | 0.994986         | 10.40251       | 0.0         | 67.60x   |
| 2        | 20480     | float32   | 1114.38            | 0.998606         | 32.41801       | 0.0         | 34.38x   |
| 2        | 20480     | float64   | 1944.97            | 0.998749         | 63.25174       | 0.0         | 30.75x   |
| 3        | 5120      | float32   | 91.18              | 0.000001         | 0.59898        | 0.0         | 152.23x  |
| 3        | 5120      | float64   | 1237.69            | 0.000000         | 1.19471        | 0.0         | 1035.97x |
| 3        | 10240     | float32   | 596.66             | 0.000002         | 4.08636        | 0.0         | 146.01x  |
| 3        | 10240     | float64   | 8603.12            | 0.000000         | 8.46728        | 0.0         | 1016.04x |
| 3        | 20480     | float32   | 4764.25            | 0.000002         | 31.27813       | 0.0         | 152.32x  |
| 3        | 20480     | float64   | 70084.94           | 0.000000         | 65.37638       | 0.0         | 1072.02x |
| 4        | 5120      | float32   | 0.63               | 0.000001         | N/A            | N/A         | N/A      |
| 4        | 5120      | float64   | 0.001              | 0.000000         | N/A            | N/A         | N/A      |
| 4        | 10240     | float32   | 0.001              | 0.000002         | N/A            | N/A         | N/A      |
| 4        | 10240      | float64   | 0.001               | 0.000000         | N/A            | N/A         | N/A      |
| 4        | 20480     | float32   | 0.002              | 0.000000         | N/A            | N/A         | N/A      |
| 4        | 20480    | float64   | 0.002              | 0.000000        | N/A            | N/A         | N/A      |

